### Part 1: Cluster Analysis

KMeans Implementation

Hierarchical Analysis

### Part 2: Text Mining

Count Vector Implementation

 Tfidf Vector Implementation

### Part 3:  Artificial Neural Network (ANN)

ANN Implementation